In [1]:
import ollama
import json
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

/opt/anaconda3/envs/ai_diary/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
! ollama list

NAME                       ID              SIZE      MODIFIED     
nomic-embed-text:latest    0a109f422b47    274 MB    9 hours ago     
deepseek-r1:8b             28f8fd6cdc67    4.9 GB    9 hours ago     
llama3.2:latest            a80c4f17acd5    2.0 GB    10 hours ago    


In [21]:
response = ollama.chat(
    model = "llama3.2" , 
    messages = [
        {"role": "user" , "content": "What's the capital of Germany?"}
    ] , 
    # temperature = 0.0 , 
    # top_p = 0.8 , 
    # max_tokens = 2000
)

In [22]:
print(response["message"]["content"])

The capital of Germany is Berlin.


In [4]:
def get_chatbot_response(model_name , messages):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"] , "content": message["content"]})
    
    response = ollama.chat(
        model = model_name , 
        messages=input_messages , 
    )

    return response["message"]["content"]



## Get LLM Response

In [5]:
model_name = "llama3.2"
messages = [{"role": "user" , "content": "What's the capital of Germany?"}]
response = get_chatbot_response(model_name=model_name , messages=messages)
print(response)

The capital of Germany is Berlin.


## Prompt Engineering

### Structured output

In [34]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one bellow. you are not allowed to write anything other than the json object:
[
{
   "country": the country that you will get the capital of
   "capital": the capital of the country stated
}
]
"""

messages = [{"role": "system" , "content":system_prompt}]
messages.append({"role":"user" , "content":"What's the capital of Germany?"})
response = get_chatbot_response(model_name=model_name , messages=messages)
print(response)

[
  {
    "country": "Germany",
    "capital": "Berlin"
  }
]


In [36]:
type(response)

str

In [37]:
json_response = json.loads(response)
json_response

[{'country': 'Germany', 'capital': 'Berlin'}]

In [40]:
type(json_response[0]) , json_response[0]["capital"]

(dict, 'Berlin')

## Input Structure

In [41]:
user_input = """
Get me the capitals of the following countries:
'''
1. Italy
2. Germany
3. France
''' 
"""

messages = [{"role": "system" , "content": system_prompt}]
messages.append({"role":"user" , "content": user_input})
response = get_chatbot_response(model_name=model_name , messages=messages)

In [42]:
print(response)

[
  {
    "country": "Italy",
    "capital": "Rome"
  },
  {
    "country": "Germany",
    "capital": "Berlin"
  },
  {
    "country": "France",
    "capital": "Paris"
  }
]


In [43]:
json_response = json.loads(response)

In [44]:
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'Germany', 'capital': 'Berlin'},
 {'country': 'France', 'capital': 'Paris'}]

### Give the model time to think (Chain of Thought)

In [45]:
user_prompt = """ 
Calculate the result of this equation: 1 + 3

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
   result: The final number resulted from calculating the equation above
}
"""

messages = [{"role": "user" , "content": user_prompt}]
response = get_chatbot_response(model_name=model_name , messages=messages)
print(response)

{
    "result": 4
}


In [46]:
259/2*8654+91072*33-12971

4113098.0

In [49]:
user_prompt = """ 
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
   result: The final number resulted from calculating the equation above
}
"""

messages = [{"role": "user" , "content": user_prompt}]
response = get_chatbot_response(model_name=model_name , messages=messages)
print(response)


{
  "result": -10132777.857142856 }


In [52]:
user_prompt = """ 
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{  steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to the final result.Feel free to write in free text.
   result: The final number resulted from calculating the equation above
}
"""


messages = [{"role": "user" , "content": user_prompt}]
response = get_chatbot_response(model_name=model_name , messages=messages)
print(response)


{
  "steps": [
    "Start with the division: 259/2 = 129.5",
    "Multiply 129.5 by 8654: 129.5*8654 = 1132939",
    "Multiply 91072 by 33: 91072*33 = 3004068",
    "Add 1132939 and 3004068: 1132939 + 3004068 = 4137007",
    "Subtract 12971 from 4137007: 4137007 - 12971 = 4128636"
  ],
  "result": 4128636
}


## RAG : Retrieval Augmented Generation

In [6]:
user_prompt = """
What's new in iphone 16? 
"""

messages = [{"role":"user" , "content":user_prompt}]
response = get_chatbot_response(model_name=model_name , messages=messages)
print(response)

I'm not aware of any official information about an iPhone 16, as Apple has not officially announced a 16th-generation iPhone. However, I can provide you with some rumors and leaks that have been circulating:

* Rumored features:
	+ Improved cameras with a new periscope lens for better zoom capabilities
	+ Enhanced display with higher refresh rates (up to 120Hz) and faster charging
	+ New colors and design options, potentially including a titanium frame
	+ Enhanced biometric security features, such as an in-display fingerprint sensor or Face ID 2.0
	+ Possibly a new processor from Apple A17 Bionic chip, which would provide improved performance and power efficiency
* Release date:
	+ Rumored to be released in the fall of 2023, potentially around September or October
	+ However, please note that this is just speculation and has not been officially confirmed by Apple

Please keep in mind that these are just rumors and leaks, and Apple may choose to surprise us with new features and designs

In [7]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [8]:
user_prompt = f"""
{iphone_16}
What's new in iphone 16?
"""

messages = [{"role":"user" , "content":user_prompt}]
response = get_chatbot_response(model_name , messages)
print(response)

Here are the key updates in the iPhone 16:

1. Display: Larger screens for different models:
	* Base model: 6.1-inch display
	* iPhone 16 Plus: 6.7-inch screen
	* iPhone 16 Pro and Pro Max: 6.3 and 6.9 inches respectively

2. Design: Thinner bezels, more durable Ceramic Shield on the front.

3. Performance:
	* Powered by new A18 chip (A18 Pro for Pro models) with improved performance.
	* Enhanced neural engine capabilities, faster GPU for gaming and machine learning tasks.

4. Camera:
	* Dual-camera setup with a 48MP main sensor on the base model.
	* 48MP Ultra Wide and 5x telephoto camera on Pro models.

5. Audio: Advanced audio features like "Audio Mix" for better audio capture during video recording.

6. Battery Life: Extended battery life, especially in the iPhone 16 Pro Max.

7. Charging: Switched to USB-C for faster charging and data transfer.

8. Video Encoding: Supports up to 2x faster video encoding on Pro models.

9. Pricing:
	* Starting at $799 for the base model.
	* Startin

### Automatically Extract context data from database

In [9]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [10]:
data = [iphone_16 , samsung_s23]

In [27]:
user_prompt = "What's new in samsumg s23?"

In [12]:

model = SentenceTransformer("hkunlp/instructor-xl")
sentences = ["What is the best embedding model?", "Llama 3.2 is great!"]
embeddings = model.encode(sentences)
print(embeddings.shape)  # Output shape: (2, 768)

(2, 768)


In [28]:
user_prompt_embeddings = model.encode(user_prompt)

In [18]:
import numpy as np

def get_embedding(model, text_input):
    """
    Generate an embedding similar to OpenAI's structure.
    Ensures output is a 2D array.
    """
    embeddings = model.encode(text_input)

    # Ensure the output is a 2D array (1 sample, N features)
    return np.array(embeddings).reshape(1, -1)

In [19]:
data_embeddings = [get_embedding(model , text)[0] for text in data]


In [16]:
len(data_embeddings)

2

In [29]:
data_similarity = cosine_similarity([user_prompt_embeddings] , data_embeddings)

In [30]:
data_similarity

array([[0.69005114, 0.82040536]], dtype=float32)

In [31]:
closest_entry_index = data_similarity.argmax()
closest_entry_index

1

In [32]:
data[closest_entry_index]

"\nThe Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra\u200b\nSTUFF\n\nTECHRADAR\n.\n\nIn terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops\u200b\nTECHRADAR\n.\n\nThe S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also inclu

In [33]:
user_prompt_with_data = f""" 
{data[closest_entry_index]}

{user_prompt}
"""

In [34]:
print(user_prompt_with_data)

 

The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity too

In [35]:
messages = [{"role":"user" , "content":user_prompt_with_data}]
response = get_chatbot_response(model_name , messages)
print(response)

Here are the key updates in Samsung S23:

1. Processor: Snapdragon 8 Gen 2 processor for enhanced performance, especially for gaming and multitasking.
2. Design: Streamlined camera module with a cleaner look, same 6.1-inch 120Hz AMOLED display, and tougher Gorilla Glass Victus 2 for better protection.
3. Camera:
   - S23 Ultra has a 200MP main camera offering impressive photo clarity in low-light conditions.
   - Selfie camera is updated to a 12MP sensor for sharper selfies.
4. Battery: Solid battery life, with the S23 Ultra featuring a 5000mAh battery and 45W wired charging.
5. Additional feature (only on S23 Ultra): Productivity tool like the S-Pen for note-taking and creative tasks.

Overall, the Samsung S23 series offers enhanced performance, durability, and camera quality, making it a strong contender in the market.
